<a href="https://colab.research.google.com/github/Geek-ubaid/clustering-analysis/blob/master/Experiment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Lab Experiment 5**

**Name : Ubaid Usmani**<br>
**Reg No: 17BBCE0983**

#### **Question**

Download a dataset from UCI repository of any application implement the following algorithm
- K-mode clustering
- Gaussian Mixture Model Using the Expectation Maximization
- Compare the performances of above and Give your Inferences
---

**Table of content:**

1. <a href="#kmeans">K-mode clustering with scratch</a>
2. <a href="https://colab.research.google.com/drive/1Kx3fpg-GajByXAVI7TCB8qIb6NV4blug#scrollTo=tKeG-1XGjlTt&line=1&uniqifier=1">Gaussian Mixture model using the EM algorithm</a> 
3. <a href="">Comparing results of both the model</a>

---


## **About Dataset**


The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

<br/>

#### **Data set used**

bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]

<br/>

#### **Attributes**

**Input Variables (features)**

1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
7. loan: has personal loan? (categorical: 'no','yes','unknown')
8. contact: contact communication type (categorical: 'cellular','telephone')
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration: last contact duration, in seconds (numeric). 
12. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
16. emp.var.rate: employment variation rate - quarterly indicator (numeric)
17. cons.price.idx: consumer price index - monthly indicator (numeric)
18. cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19. euribor3m: euribor 3 month rate - daily indicator (numeric)
20. nr.employed: number of employees - quarterly indicator (numeric)

**Output variable (desired target)**

- y - has the client subscribed a term deposit? (binary: 'yes','no')

#### **Reference Link**
https://archive.ics.uci.edu/ml/datasets/bank+marketing


## **Data Preprocessing**

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
dataset_path = '/content/drive/My Drive/ML Experiments/bankmarketing.csv'
data = pandas.read_csv(dataset_path)

label =data.iloc[:,-1]

data_numeric = data.loc[:,data.dtypes!=np.object]
print(data_numeric.head())

data_cat = data.loc[:,data.dtypes==np.object]
print(data_cat.head())

   age  duration  campaign  ...  cons.conf.idx  euribor3m  nr.employed
0   56       261         1  ...          -36.4      4.857       5191.0
1   57       149         1  ...          -36.4      4.857       5191.0
2   37       226         1  ...          -36.4      4.857       5191.0
3   40       151         1  ...          -36.4      4.857       5191.0
4   56       307         1  ...          -36.4      4.857       5191.0

[5 rows x 10 columns]
         job  marital    education  default  ... month day_of_week     poutcome   y
0  housemaid  married     basic.4y       no  ...   may         mon  nonexistent  no
1   services  married  high.school  unknown  ...   may         mon  nonexistent  no
2   services  married  high.school       no  ...   may         mon  nonexistent  no
3     admin.  married     basic.6y       no  ...   may         mon  nonexistent  no
4   services  married  high.school       no  ...   may         mon  nonexistent  no

[5 rows x 11 columns]


### **Correlation Analysis for feature selection**

In [0]:
from scipy import stats

def chi_square_test(feature1, feature2):
  result = []
  result.append(feature1)

  contingency_table = pd.crosstab(data_cat[feature1], data_cat[feature2])
  observed_values = contingency_table.values
  chi_square_result = stats.chi2_contingency(contingency_table)
  expected_values = chi_square_result[3]

  ## calculate degree of freedom ( more the degree more chances of getting good results)

  no_rows = len(contingency_table.iloc[0:2,0])
  no_columns = len(contingency_table.iloc[0,0:2])

  degree_freedom = (no_rows-1) / (no_columns-1)
  result.append(degree_freedom)

  ## Significance level (Generally used for the test)
  alpha=0.05
  
  chi_square = sum([(o-e)**2./e for o,e in zip(observed_values, expected_values)])
  chi_square_statistic = chi_square[0] + chi_square[1]
  result.append(chi_square_statistic)

  critical_value = stats.chi2.ppf(q = 1-alpha, df=degree_freedom)
  result.append(critical_value)

  p_value = 1 - stats.chi2.cdf(x = chi_square_statistic, df = degree_freedom)
  result.append(p_value) 

  ## Check for Null hypothesis using p_value and chi_square statistics

  if chi_square_statistic >= critical_value or p_value <= alpha:
    result.append('True')
    return result
  else:
    result.append('False')
    return result

In [0]:
corr_cat = []
for i in data_cat.columns.tolist()[:-1]:
  corr_cat.append(chi_square_test(i,'y'))

cat_corr = pd.DataFrame(corr_cat, columns=['Feature' , 'DOF', 'Chi_Sq-Stats', 'critical_val', 'p_value', 'Correlated'])

#### **Correlation of all categorical variable with the target variable 'y'**

In [0]:
cat_corr

,Feature,DOF,Chi_Sq-Stats,critical_val,p_value,Correlated
0,job,1.0,961.242440,3.841459,0.000000e+00,True
1,marital,1.0,122.655152,3.841459,0.000000e+00,True
2,education,1.0,193.105905,3.841459,0.000000e+00,True
3,default,1.0,406.577515,3.841459,0.000000e+00,True
4,housing,1.0,5.684496,3.841459,1.711546e-02,True
5,loan,1.0,1.094028,3.841459,2.955806e-01,False
6,contact,1.0,863.269081,3.841459,0.000000e+00,True
7,month,1.0,3101.149351,3.841459,0.000000e+00,True
8,day_of_week,1.0,26.144939,3.841459,3.167261e-07,True
9,poutcome,1.0,4230.523798,3.841459,0.000000e+00,True


So leaving loan all other variable passed the correlation test and can be used in clustering.

In [0]:
replace_map = {'y': {'yes': 1, 'no': 0}}
data.replace(replace_map, inplace=True)
data['age_bin'] = pandas.cut(data['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                            labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])

data  = data.drop('age',axis = 1)

features = data[['age_bin','job', 'marital', 'education',\
              'default', 'housing','contact',\
              'month','day_of_week','poutcome','y']]

le = LabelEncoder()
features_encoded = features.apply(le.fit_transform)


label=data[['y']]



<a id="kmeans"></a>

## **K-modes Clustering (Without library implementation)**

K-Modes algorithm is a clustering used to cluster data based on dissimilarity among the cluster data. The smaller the number more chances are there of them being in the same cluster. Here the mode function is used to cluster data. A mode is a vector of elements that minimizes the dissimilarities between the vector itself and each object of the data. We will have as many modes as the number of clusters we required, since they act as centroids.
<p align='center'>
<img src='https://i.ytimg.com/vi/KK4wfiQqUN0/maxresdefault.jpg'/>
</p>


### **Packages Importing**

In [0]:
## Importing packages
import warnings
warnings.filterwarnings('ignore')

import random 
from collections import defaultdict, Counter, deque

import numpy as np
import matplotlib.pyplot as plt  
import pandas
from scipy import stats
from sklearn import preprocessing
from sklearn.metrics import classification_report

### **Utils Functions**

In [0]:

def pickup_centroids(df,k):
    centroids_idx = np.random.choice(a = df.index.values, replace = False, size = k)
    centroid_a = df.loc[centroids_idx[0]].values
    centroid_b = df.loc[centroids_idx[1]].values
    return(centroid_a,centroid_b)

def distance_mismatch(a,b):
    return (a != b).sum()

def compute_distances_to_centroids(centroid_a,centroid_b,df):
    
    dic_distances = {}
    for idx,row in df.iterrows():
        candidat = row.values
        distance_to_a = distance_mismatch(candidat[:-1],centroid_a[:-1])
        distance_to_b = distance_mismatch(candidat[:-1],centroid_b[:-1])
        affectation = np.argmin([distance_to_a,distance_to_b])
        dic_distances[idx] = {"distance_to_a" : distance_to_a,
                              "distance_to_b" : distance_to_b,
                              "cluster" : affectation}
        
    return dic_distances


def extract_assigned_data(dic_distances,df):
    c1list = []
    c2list = []
    for k,v in dic_distances.items():
        if v["cluster"] == 0:
            c1list.append(df.loc[k].values)
        else:
            c2list.append(df.loc[k].values)

    if len(c1list)>0:
        a = np.vstack(c1list)
    else:
        a = np.array([])
        
    if len(c2list)>0:
        b = np.vstack(c2list)
    else:
        b = np.array([])
        
    return a,b


def compute_mode(array):
    return stats.mode(array)[0][0]


def compute_performance(dic_distances):
    
    distances_list = []
    for k,v in dic_distances.items():
        if v["cluster"] == 0:
            distances_list.append(v["distance_to_a"])
        else:
            distances_list.append(v["distance_to_b"])
    return (np.array(distances_list)).sum()


## **Main Algorithm**

In [0]:
%%timeit
def kmodes(df,k=2,threshold=1,iterations=5, verbose = True):
   
  dic_results = {}  
  centroid_a,centroid_b = pickup_centroids(df,k)

  for i in range(iterations):

    if verbose:
      print("iteration : ",i)

      dic_distances = compute_distances_to_centroids(centroid_a,centroid_b,df)
      array_a,array_b = extract_assigned_data(dic_distances,df)
      mycdt = len(array_a)==0 or len(array_b)==0

      if mycdt:
        continue

      futur_centroid_a = compute_mode(array_a)
      futur_centroid_b = compute_mode(array_b)

      print('iteration : ' + str(i) + ' - complete')

      d = distance_mismatch(futur_centroid_a,centroid_a) + distance_mismatch(futur_centroid_b,centroid_b)

      if verbose:
        print("distance between present and new centroids : ", d)

      if d<threshold:
        break
      
      centroid_a = futur_centroid_a
      centroid_b = futur_centroid_b
                 
  print("Total elements in cluster 1 " + str(len(array_a)))
  print("Total elements in cluster 2 " + str(len(array_b)))

  df_pred = df.copy()
  df_pred['pred_cluster'] = pandas.Series(np.random.randn(len(df)), index=df_pred.index)
  for idx,row in df_pred.iterrows():
    df_pred.iloc[idx,-1] = dic_distances[idx]['cluster']

  return df_pred

  
def main():

  df = kmodes(features, verbose = True)
  print(df.head())
  return df

kmodes_clsuter_df = main()
kmodes_clsuter_df['y'] = kmodes_clsuter_df['y'].astype('float16')

iteration :  0
iteration : 0 - complete
distance between present and new centroids :  5
iteration :  1
iteration : 1 - complete
distance between present and new centroids :  0
Total elements in cluster 1 22562
Total elements in cluster 2 18626
  age_bin        job  marital  ...     poutcome  y pred_cluster
0   50-60  housemaid  married  ...  nonexistent  0          0.0
1   50-60   services  married  ...  nonexistent  0          0.0
2   30-40   services  married  ...  nonexistent  0          0.0
3   30-40     admin.  married  ...  nonexistent  0          0.0
4   50-60   services  married  ...  nonexistent  0          0.0

[5 rows x 12 columns]
iteration :  0
iteration : 0 - complete
distance between present and new centroids :  6
iteration :  1
iteration : 1 - complete
distance between present and new centroids :  0
Total elements in cluster 1 26726
Total elements in cluster 2 14462
  age_bin        job  marital  ...     poutcome  y pred_cluster
0   50-60  housemaid  married  ...  nonex

In [0]:
train_accuracy = np.mean(kmodes_clsuter_df['pred_cluster'].ravel() == kmodes_clsuter_df['y'].ravel()) * 100

print('train accuracy : ' , train_accuracy)

print(classification_report(kmodes_clsuter_df['pred_cluster'], kmodes_clsuter_df['y']))

train accuracy :  59.71885015052928
              precision    recall  f1-score   support

         0.0       0.61      0.91      0.73     24497
         1.0       0.51      0.14      0.22     16691

    accuracy                           0.60     41188
   macro avg       0.56      0.52      0.48     41188
weighted avg       0.57      0.60      0.52     41188



<a id="gmm"></a>
## **Gaussian mixture model using EM**

Gaussian Mixture Models (GMMs) assume that there are a certain number of Gaussian distributions, and each of these distributions represent a cluster. Hence, a Gaussian Mixture Model tends to group the data points belonging to a single distribution together.
<br/>

<p align='center'>
<img src='https://www.researchgate.net/profile/Jeremie_Sublime/publication/310644322/figure/fig4/AS:431286679543811@1479838165688/Illustration-of-the-EM-algorithm-GMM-on-the-Old-Faithful-data-set.png'/ height=50% width=50%>
</p>

## **Importing Packages**

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import matplotlib as mpl
import pandas as pd
from sklearn import preprocessing, mixture
from sklearn.metrics import classification_report

## **Main Algorithm**



In [0]:
class GaussianMixModel(object):
    def __init__(self, X, k=2):
        # Algorithm can work for any number of columns in dataset
        X = np.asarray(X)
        self.m, self.n = X.shape
        self.data = X.copy()
        print (np.mean(X))
        # number of mixtures
        self.k = k

    def _init(self):
        # init mixture means/sigmas
        self.mean_arr = np.asmatrix(np.random.random((self.k, self.n))+np.mean(self.data))
        #self.mean_arr[0]=0;
        #self.mean_arr[1]=25;
        self.sigma_arr = np.array([np.asmatrix(np.identity(self.n)) for i in range(self.k)])
        self.phi = np.ones(self.k)/self.k
        self.Z = np.asmatrix(np.empty((self.m, self.k), dtype=float))
        #Z Latent Variable giving probability of each point for each distribution

    def fit(self, tol=1e-4):
        # Algorithm will run unti max of log-likelihood is achieved
        self._init()
        num_iters = 0
        logl = 1
        previous_logl = 0
        while(logl-previous_logl > tol):
            previous_logl = self.loglikelihood()
            self.e_step()
            self.m_step()
            num_iters += 1
            logl = self.loglikelihood()
            print('Iteration %d: log-likelihood is %.6f'%(num_iters, logl))
        print('Terminate at %d-th iteration:log-likelihood is %.6f'%(num_iters, logl))

    def loglikelihood(self):
        logl = 0
        for i in range(self.m):
            tmp = 0
            for j in range(self.k):
                #print(self.sigma_arr[j])
                tmp += sp.multivariate_normal.pdf(self.data[i, :],self.mean_arr[j, :].A1,self.sigma_arr[j, :]) * self.phi[j]
            logl += np.log(tmp)
        return logl




    def e_step(self):
        #Finding probability of each point belonging to each pdf and putting it in latent variable Z
        for i in range(self.m):
            den = 0
            for j in range(self.k):
                #print (self.data[i, :])
                num = sp.multivariate_normal.pdf(self.data[i, :],
                                                       self.mean_arr[j].A1,
                                                       self.sigma_arr[j]) *\
                      self.phi[j]
                den += num

                self.Z[i, j] = num
            self.Z[i, :] /= den
            assert self.Z[i, :].sum() - 1 < 1e-4  # Program stop if this condition is false

    def m_step(self):
        #Updating mean and variance
        for j in range(self.k):
            const = self.Z[:, j].sum()
            self.phi[j] = 1/self.m * const
            _mu_j = np.zeros(self.n)
            _sigma_j = np.zeros((self.n, self.n))
            for i in range(self.m):
                _mu_j += (self.data[i, :] * self.Z[i, j])
                _sigma_j += self.Z[i, j] * ((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))

            self.mean_arr[j] = _mu_j / const
            self.sigma_arr[j] = _sigma_j / const


## **Testing and visualizing GMM**

In [0]:
def cluster_gmm(No_Component=2):
  
    gmm = mixture.GaussianMixture(n_components=2)
    gmm.fit(features_encoded)
    
    y_train_pred = gmm.predict(features_encoded)

    print(classification_report(y_train_pred, label))

    return gmm, y_train_pred
  


In [0]:
%%timeit
gmm_model, gmm_preds = cluster_gmm(2)

gmm_df = features.copy()

gmm_df['preds'] = gmm_preds


              precision    recall  f1-score   support

           0       0.48      0.90      0.63     19303
           1       0.60      0.13      0.21     21885

    accuracy                           0.49     41188
   macro avg       0.54      0.52      0.42     41188
weighted avg       0.55      0.49      0.41     41188

              precision    recall  f1-score   support

           0       0.48      0.90      0.63     19303
           1       0.60      0.13      0.21     21885

    accuracy                           0.49     41188
   macro avg       0.54      0.52      0.42     41188
weighted avg       0.55      0.49      0.41     41188

              precision    recall  f1-score   support

           0       0.52      0.87      0.65     21850
           1       0.40      0.10      0.15     19338

    accuracy                           0.51     41188
   macro avg       0.46      0.48      0.40     41188
weighted avg       0.46      0.51      0.42     41188

              preci

## Model Comparison (GMM vs KModes) over Bank Marketing dataset 

#### **Classification report of GMM**

In [0]:
print(classification_report(gmm_preds, label))

              precision    recall  f1-score   support

           0       0.52      0.87      0.65     21850
           1       0.40      0.10      0.15     19338

    accuracy                           0.51     41188
   macro avg       0.46      0.48      0.40     41188
weighted avg       0.46      0.51      0.42     41188



#### **Classification report of KModes**

In [0]:
print(classification_report(kmodes_clsuter_df['pred_cluster'], kmodes_clsuter_df['y']))

              precision    recall  f1-score   support

         0.0       0.61      0.91      0.73     24497
         1.0       0.51      0.14      0.22     16691

    accuracy                           0.60     41188
   macro avg       0.56      0.52      0.48     41188
weighted avg       0.57      0.60      0.52     41188



From the above results we can clearly see that the KModes algorithm is working better in clustering catrgorical data when the same amount of data is used to fit both the models.

The final inference we get after performing all the operations are:
- Kmodes work best with clustering categorical data
- Kmodes is slow as compared to GMM
- GMM gives bad result when clustering only with categorical inputs
- Correlation and features selection plays a vital role in GMM and Kmodes both